<h1 align=center><font size = 5><em></em>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Importing the needed Libraries for this project  

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import xml 
 
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from bs4 import BeautifulSoup #library for web scraping

print('Libraries imported.')



Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge
Libraries imported.


## Define the URL where we will extract data about Toronto Neighborhoods 

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
Canada_data = BeautifulSoup(source, 'lxml')

In [3]:
# create a new Dataframe
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

# loop through to find postcode, borough, neighborhood 
content = Canada_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

# clean dataframe 
toronto = toronto[toronto.Borough!='Not assigned']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
                                 
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Cleaning the DataSet 

In [4]:
# We only process the rows that have an assigned borough. So we will drop rows with a borough that is Not assigned.
df = df.dropna()
empty = 'Not assigned'
df = df[(df.Postalcode != empty ) & (df.Borough != empty) & (df.Neighborhood != empty)]

In [5]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [7]:
print(df.shape)
df.head(12)  # The DataFrame below represents the table about Toronto Neighborhood on the Wikipedia website
# Now, in order to segment and clustering Neighborhood in Toronto, we have to retrieve locations of Neighborhood in order to have a final 
# DataSet to begin our exploration.

(103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


## Add the latitude and longitude coordinates to the dataframe

In [8]:
# Install and import Geocoder library
!pip install geocoder
import geocoder
print( 'geocoder imported !')

Requirement not upgraded as not directly required: geocoder in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages
Requirement not upgraded as not directly required: click in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: future in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: ratelim in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from geocoder)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->geocoder)
Requirement not upgraded as not directly required: i

In [9]:
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords
    
get_latlng('M4G')

[43.70976500000006, -79.36379132299999]

In [10]:
# Postal codes coordinates
postal_codes = df['Postalcode']    
coords = [ get_latlng(postal_code) for postal_code in postal_codes.tolist() ]

In [11]:
# Now, we are going to add Latitude and Longitude columns to our DataFrame 
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [12]:
# Lets see the first ten rows of our new DataFrame 

df.head(10)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811650,-79.195561
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785605,-79.158701
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175299
3,M1G,Scarborough,Woburn,43.768216,-79.217610
4,M1H,Scarborough,Cedarbrae,43.769608,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743085,-79.232172
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726260,-79.263670
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713213,-79.284910
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.723575,-79.234976
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696690,-79.260069


In [13]:
# Verify the number of boroughs and neighborhoods in our DataSet

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


## Use geopy library to get the latitude and longitude values of Toronto

In [14]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


## Create a map of Toronto with neighborhoods superimposed on top

In [15]:
# create a map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

For illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Scarborough (East Toronto). So let's slice the original dataframe and create a new dataframe of the Scarborough data.

In [16]:
Scarborough_data = df[df['Borough'] == 'Scarborough'].reset_index(drop=True)
Scarborough_data.head(10)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811650,-79.195561
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785605,-79.158701
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175299
3,M1G,Scarborough,Woburn,43.768216,-79.217610
4,M1H,Scarborough,Cedarbrae,43.769608,-79.239440
5,M1J,Scarborough,Scarborough Village,43.743085,-79.232172
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.726260,-79.263670
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713213,-79.284910
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.723575,-79.234976
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696690,-79.260069


Let's get the geographical coordinates of Scarborough.

In [17]:
address = 'Scarborough, Toronto'

geolocator = Nominatim(user_agent="Scarborough_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough, Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough, Toronto are 43.773077, -79.257774.


As we did with all of Toronto City, let's visualize Scarborough neighborhoods in it.

In [18]:
# create map of Scarborough using latitude and longitude values
map_scarborough = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Scarborough_data['Latitude'], Scarborough_data['Longitude'], Scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_scarborough)  
    
map_scarborough

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [19]:
CLIENT_ID = 'NTQRLDKTMVYHOOKBKD2LOYHMJDRNRB40Q1QQB30PUQPINURO' # your Foursquare ID
CLIENT_SECRET = 'OAEHTXDYPJCOG5MHO400OLP02ZEP55AQQVLB5OJMOM3GFF2O' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: NTQRLDKTMVYHOOKBKD2LOYHMJDRNRB40Q1QQB30PUQPINURO
CLIENT_SECRET:OAEHTXDYPJCOG5MHO400OLP02ZEP55AQQVLB5OJMOM3GFF2O


Let's explore the first neighborhood in our dataframe

Get the neighborhood's name.

In [20]:
Scarborough_data.loc[0, 'Neighborhood']

'Rouge, Malvern'

Get the neighborhood's latitude and longitude values

In [21]:
neighborhood_latitude = Scarborough_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Scarborough_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = Scarborough_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Rouge, Malvern are 43.81165000000004, -79.19556138899998.


#### Now, let's get the top 100 venues that are in Rouge, Malvern within a radius of 500 meters.

First, let's create the GET request URL. Name your URL url

In [22]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=NTQRLDKTMVYHOOKBKD2LOYHMJDRNRB40Q1QQB30PUQPINURO&client_secret=OAEHTXDYPJCOG5MHO400OLP02ZEP55AQQVLB5OJMOM3GFF2O&v=20180605&ll=43.81165000000004,-79.19556138899998&radius=500&limit=100'

Send the GET request and examine the results

In [23]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c602826351e3d194f479efd'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-5c0af1c44a1cc0002cc5537b-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/airport_rentalcar_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d1ef941735',
         'name': 'Rental Car Location',
         'pluralName': 'Rental Car Locations',
         'primary': True,
         'shortName': 'Rental Car'}],
       'id': '5c0af1c44a1cc0002cc5537b',
       'location': {'address': '10 Thornmont Drive',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'distance': 486,
        'formattedAddress': ['10 Thornmont Drive',
         'Toronto ON M1B 3J4',
         'Canada'],
        'labeledLatLngs': [{'label': 'display',


Now, let's extract the category of each venue 

In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [25]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Scarborough Bad Credit Car Loans,Rental Car Location,43.808123,-79.199133
1,Canadian Appliance Source Whitby,Home Service,43.808353,-79.191331


It seems to this Neighborhood don't have many top venues. Actually, only 2 

## Explore Neighborhoods in Scarborough

#### Let's create a function to repeat the same process to all the neighborhoods in Scarborough

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Code to run the above function on each neighborhood and create a new dataframe called *Scarborough_venues*.

In [27]:
Scarborough_venues = getNearbyVenues(names=Scarborough_data['Neighborhood'],
                                   latitudes=Scarborough_data['Latitude'],
                                   longitudes=Scarborough_data['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge


Let's check the size of the resulting dataframe

In [28]:
print(Scarborough_venues.shape)
Scarborough_venues.head()

(97, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.811650,-79.195561,Scarborough Bad Credit Car Loans,43.808123,-79.199133,Rental Car Location
1,"Rouge, Malvern",43.811650,-79.195561,Canadian Appliance Source Whitby,43.808353,-79.191331,Home Service
2,"Highland Creek, Rouge Hill, Port Union",43.785605,-79.158701,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Guildwood, Morningside, West Hill",43.765690,-79.175299,The Strawberry Patch,43.764738,-79.173081,Tea Room
4,"Guildwood, Morningside, West Hill",43.765690,-79.175299,Homestead Roofing Repair,43.765140,-79.178663,Construction & Landscaping


Let's check how many venues were returned for each neighborhood

In [29]:
Scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,17,17,17,17,17,17
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Birch Cliff, Cliffside West",6,6,6,6,6,6
Cedarbrae,1,1,1,1,1,1
"Clairlea, Golden Mile, Oakridge",10,10,10,10,10,10
"Clarks Corners, Sullivan, Tam O'Shanter",12,12,12,12,12,12
"Cliffcrest, Cliffside, Scarborough Village West",11,11,11,11,11,11
"Dorset Park, Scarborough Town Centre, Wexford Heights",3,3,3,3,3,3
"East Birchmount Park, Ionview, Kennedy Park",4,4,4,4,4,4


### Let's find out how many unique categories can be curated from all the returned venues

In [30]:
print('There are {} uniques categories.'.format(len(Scarborough_venues['Venue Category'].unique())))

There are 56 uniques categories.


## Analyze Each Neighborhood

In [31]:
# one hot encoding
Scarborough_onehot = pd.get_dummies(Scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Scarborough_onehot['Neighborhood'] = Scarborough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Scarborough_onehot.columns[-1]] + list(Scarborough_onehot.columns[:-1])
Scarborough_onehot = Scarborough_onehot[fixed_columns]

Scarborough_onehot.head()

,Neighborhood,Auto Garage,Badminton Court,Bakery,Bar,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hobby Shop,Home Service,Indian Restaurant,Intersection,Japanese Restaurant,Korean Restaurant,Liquor Store,Metro Station,Other Great Outdoors,Park,Pharmacy,Pizza Place,Playground,Pool,Rental Car Location,Restaurant,Sandwich Place,Shanghai Restaurant,Shipping Store,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Train Station,Vietnamese Restaurant,Wings Joint
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Highland Creek, Rouge Hill, Port Union",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,"Guildwood, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size

In [32]:
Scarborough_onehot.shape

(97, 57)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [33]:
Scarborough_grouped = Scarborough_onehot.groupby('Neighborhood').mean().reset_index()
Scarborough_grouped

,Neighborhood,Auto Garage,Badminton Court,Bakery,Bar,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Business Service,Chinese Restaurant,Coffee Shop,College Stadium,Construction & Landscaping,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Furniture / Home Store,General Entertainment,Gift Shop,Golf Course,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hobby Shop,Home Service,Indian Restaurant,Intersection,Japanese Restaurant,Korean Restaurant,Liquor Store,Metro Station,Other Great Outdoors,Park,Pharmacy,Pizza Place,Playground,Pool,Rental Car Location,Restaurant,Sandwich Place,Shanghai Restaurant,Shipping Store,Shopping Mall,Skating Rink,Soccer Field,Supermarket,Sushi Restaurant,Tea Room,Thai Restaurant,Train Station,Vietnamese Restaurant,Wings Joint
0,Agincourt,0.0,0.058824,0.058824,0.0,0.058824,0.000000,0.0,0.0,0.000000,0.00,0.058824,0.000000,0.000000,0.00,0.058824,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.058824,0.000000,0.000000,0.0,0.058824,0.0,0.00,0.000000,0.058824,0.000000,0.117647,0.058824,0.0,0.117647,0.058824,0.00,0.000000,0.00,0.058824,0.000000
1,"Agincourt North, L'Amoreaux East, Milliken, St...",0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000
2,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.166667,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.166667,0.00,0.166667,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.166667,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.166667,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000
3,Cedarbrae,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000
4,"Clairlea, Golden Mile, Oakridge",0.0,0.000000,0.200000,0.0,0.000000,0.000000,0.2,0.1,0.000000,0.00,0.000000,0.200000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.1,0.000000,0.00,0.000000,0.1,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.1,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000
5,"Clarks Corners, Sullivan, Tam O'Shanter",0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.083333,0.00,0.083333,0.083333,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.00,0.000000,0.083333,0.0,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.166667,0.166667,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.083333,0.000000,0.0,0.000000,0.000000,0.00,0.083333,0.00,0.000000,0.000000
6,"Cliffcrest, Cliffside, Scarborough Village West",0.0,0.000000,0.000000,0.0,0.000000,0.090909,0.0,0.0,0.000000,0.00,0.000000,0.090909,0.000000,0.00,0.000000,0.090909,0.000000,0.181818,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.090909,0.0,0.000000,0.000000,0.090909,0.090909,0.0,0.000000,0.0,0.00,0.090909,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00000

#### Let's confirm the new size

In [34]:
Scarborough_grouped.shape

(16, 57)

#### Let's print each neighborhood along with the top 5 most common venues

In [35]:
num_top_venues = 5

for hood in Scarborough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Scarborough_grouped[Scarborough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                 venue  freq
0          Supermarket  0.12
1        Shopping Mall  0.12
2       Discount Store  0.06
3        Grocery Store  0.06
4  Shanghai Restaurant  0.06


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
                 venue  freq
0             Pharmacy   1.0
1          Auto Garage   0.0
2      Badminton Court   0.0
3         Intersection   0.0
4  Japanese Restaurant   0.0


----Birch Cliff, Cliffside West----
                   venue  freq
0               Gym Pool  0.17
1                    Gym  0.17
2                   Park  0.17
3           Skating Rink  0.17
4  General Entertainment  0.17


----Cedarbrae----
                 venue  freq
0           Playground   1.0
1          Auto Garage   0.0
2      Badminton Court   0.0
3         Intersection   0.0
4  Japanese Restaurant   0.0


----Clairlea, Golden Mile, Oakridge----
           venue  freq
0    Coffee Shop   0.2
1         Bakery   0.2
2       Bus Line   0.2
3   Intersectio

#### Let's put the results into a pandas dataframe

#### First, let's write a function to sort the venues in descending order

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Scarborough_grouped['Neighborhood']

for ind in np.arange(Scarborough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Scarborough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Shopping Mall,Supermarket,Shanghai Restaurant,Park,Chinese Restaurant,Grocery Store,Pool,Vietnamese Restaurant,Department Store,Discount Store
1,"Agincourt North, L'Amoreaux East, Milliken, St...",Pharmacy,Wings Joint,Gym Pool,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint
2,"Birch Cliff, Cliffside West",College Stadium,Gym,General Entertainment,Skating Rink,Park,Gym Pool,Department Store,Grocery Store,Golf Course,Gift Shop
3,Cedarbrae,Playground,Wings Joint,Gym Pool,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint
4,"Clairlea, Golden Mile, Oakridge",Bus Line,Bakery,Coffee Shop,Intersection,Soccer Field,Metro Station,Bus Station,Furniture / Home Store,Fast Food Restaurant,Fried Chicken Joint


## Cluster Neighborhoods

Run k-means to cluster the neighborhood into 4 clusters

In [38]:
# set number of clusters
kclusters = 4

Scarborough_grouped_clustering = Scarborough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Scarborough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 0, 3, 0, 0, 0, 0, 0, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [39]:
Scarborough_merged = Scarborough_data[0:16]

# add clustering labels
Scarborough_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Scarborough_merged = Scarborough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Scarborough_merged.head() # check the last columns!

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.811650,-79.195561,0,Rental Car Location,Home Service,Construction & Landscaping,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785605,-79.158701,1,Bar,Wings Joint,Department Store,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765690,-79.175299,0,Gym / Fitness Center,Construction & Landscaping,Tea Room,Park,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store
3,M1G,Scarborough,Woburn,43.768216,-79.217610,3,Korean Restaurant,Business Service,Park,Coffee Shop,Wings Joint,Discount Store,Gym,Grocery Store,Golf Course,Gift Shop
4,M1H,Scarborough,Cedarbrae,43.769608,-79.239440,0,Playground,Wings Joint,Gym Pool,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint


Finally, let's visualize the resulting clusters

In [42]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)
# add markers to the map
markers_colors = []
for lat, lon, nei , cluster in zip(Scarborough_merged['Latitude'], Scarborough_merged['Longitude'], Scarborough_merged['Neighborhood'], Scarborough_merged['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#2adddd', '#d4dd80', '#ff0000']


## Examine Clusters

#### In this final step, we are going to examine each cluster and determine the discriminating venue categories that distinguish each cluster

#### Cluster 1

In [43]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 0,Scarborough_merged.columns[[2] + list(range(5, Scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Rouge, Malvern",0,Rental Car Location,Home Service,Construction & Landscaping,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint
2,"Guildwood, Morningside, West Hill",0,Gym / Fitness Center,Construction & Landscaping,Tea Room,Park,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store
4,Cedarbrae,0,Playground,Wings Joint,Gym Pool,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint
5,Scarborough Village,0,Train Station,Grocery Store,Indian Restaurant,Restaurant,Wings Joint,Construction & Landscaping,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store
6,"East Birchmount Park, Ionview, Kennedy Park",0,Discount Store,Department Store,Coffee Shop,Wings Joint,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment
7,"Clairlea, Golden Mile, Oakridge",0,Bus Line,Bakery,Coffee Shop,Intersection,Soccer Field,Metro Station,Bus Station,Furniture / Home Store,Fast Food Restaurant,Fried Chicken Joint
8,"Cliffcrest, Cliffside, Scarborough Village West",0,Fast Food Restaurant,Wings Joint,Pizza Place,Burger Joint,Furniture / Home Store,Liquor Store,Sandwich Place,Coffee Shop,Discount Store,Pharmacy
9,"Birch Cliff, Cliffside West",0,College Stadium,Gym,General Entertainment,Skating Rink,Park,Gym Pool,Department Store,Grocery Store,Golf Course,Gift Shop
11,"Maryvale, Wexford",0,Auto Garage,Intersection,Gym Pool,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store,Fried Chicken Joint
12,Agincourt,0,Shopping Mall,Supermarket,Shanghai Restaurant,Park,Chinese Restaurant,Grocery Store,Pool,Vietnamese Restaurant,Department Store,Discount Store


#### Cluster 2

In [45]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 1,Scarborough_merged.columns[[2] + list(range(5, Scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,"Highland Creek, Rouge Hill, Port Union",1,Bar,Wings Joint,Department Store,Gym / Fitness Center,Gym,Grocery Store,Golf Course,Gift Shop,General Entertainment,Furniture / Home Store


#### Cluster 3

In [46]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 2,Scarborough_merged.columns[[2] + list(range(5, Scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,"Dorset Park, Scarborough Town Centre, Wexford ...",2,Bakery,Gift Shop,Shipping Store,Wings Joint,Construction & Landscaping,Gym,Grocery Store,Golf Course,General Entertainment,Furniture / Home Store


#### Cluster 4 

In [48]:
Scarborough_merged.loc[Scarborough_merged['Cluster Labels'] == 3,Scarborough_merged.columns[[2] + list(range(5, Scarborough_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Woburn,3,Korean Restaurant,Business Service,Park,Coffee Shop,Wings Joint,Discount Store,Gym,Grocery Store,Golf Course,Gift Shop
